In [1]:
import pandas as pd

train_url = "https://raw.githubusercontent.com/middlebury-csci-0451/CSCI-0451/main/data/palmer-penguins/train.csv"
train = pd.read_csv(train_url)

In [2]:
train.head

<bound method NDFrame.head of     studyName  Sample Number                                    Species  \
0     PAL0708             27          Gentoo penguin (Pygoscelis papua)   
1     PAL0708             22          Gentoo penguin (Pygoscelis papua)   
2     PAL0910            124        Adelie Penguin (Pygoscelis adeliae)   
3     PAL0910            146        Adelie Penguin (Pygoscelis adeliae)   
4     PAL0708             24  Chinstrap penguin (Pygoscelis antarctica)   
..        ...            ...                                        ...   
270   PAL0708              7          Gentoo penguin (Pygoscelis papua)   
271   PAL0910            121        Adelie Penguin (Pygoscelis adeliae)   
272   PAL0910             84          Gentoo penguin (Pygoscelis papua)   
273   PAL0708             29          Gentoo penguin (Pygoscelis papua)   
274   PAL0708             13        Adelie Penguin (Pygoscelis adeliae)   

     Region     Island               Stage Individual ID Clutch Compl